In [186]:
import pyrez
import datetime
from random import randint as r
from dotenv import load_dotenv
load_dotenv()
import os

In [187]:
import pyrez.api
from pyrez.api import PaladinsAPI
import pyrez.enumerations
import pyrez.models

In [188]:
api = PaladinsAPI(devId=os.getenv('DEV_ID'), authKey=os.getenv('AUTH_KEY'))
api.ping().ping

True

In [44]:
# Get match data

QUEUE = 486 # the Paladins Competitive Queue
MONTH, DAY, YEAR = ( 3, 15, 2019 ) # the date to pull match data for
HOUR = 10 # hour of play

matches = api.getMatchIdsByQueue(QUEUE, datetime.date(year=YEAR, month=MONTH, day=DAY), '10,00')
matchIds = [match.matchId for match in matches if not match.activeFlag]
matchIds

['807379762',
 '807379766',
 '807379916',
 '807379941',
 '807379965',
 '807380010',
 '807380035',
 '807380036',
 '807380133',
 '807380223',
 '807380244',
 '807380269',
 '807380285',
 '807380311',
 '807380407',
 '807380432',
 '807380439',
 '807380474',
 '807380495',
 '807380576',
 '807380618',
 '807380623',
 '807380659',
 '807380680',
 '807380697',
 '807380740',
 '807380769',
 '807380778',
 '807380786',
 '807380841',
 '807380844',
 '807380878',
 '807380897',
 '807380930',
 '807380947',
 '807381052',
 '807381058']

In [45]:
len(matchIds)

37

In [ ]:
# We need to limit our batch requests to 10 matches per
BATCHSIZE = 10

matchData = []

# from https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

for matchBatch in chunks(matchIds, 10):
    matchData += api.makeRequest("getmatchdetailsbatch", [','.join(matchBatch)])

matchData # I omitted the output here as it is pretty big

In [67]:
matchData[2]['Map_Game']

'Ranked Serpent Beach'

In [70]:
maps = list(set([match['Map_Game'] for match in matchData]))

In [71]:
maps

['Ranked Serpent Beach',
 'Ranked Frog Isle',
 'Ranked Stone Keep',
 'Ranked Jaguar Falls',
 'Ranked Fish Market',
 'Ranked Brightmarsh',
 'Ranked Frozen Guard',
 'Ranked Timber Mill',
 'Ranked Splitstone Quarry',
 'Ranked Ascension Peak',
 "Ranked Warder's Gate"]

In [82]:
matches = []
for match_id in matchIds:
    mdata = [m for m in matchData if str(m['Match']) == match_id]
    match = {
        # winners
        "team1": [m['Reference_Name'] for m in mdata if m['Win_Status'] == 'Winner'],
        # losers
        "team2": [m['Reference_Name'] for m in mdata if m['Win_Status'] != 'Winner'],
        # more data
        "map": mdata[0]['Map_Game']
    }
    matches.append(match)

matches[0]

{'map': 'Ranked Serpent Beach',
 'team1': ['Grover', 'Fernando', 'Drogoz', 'Ash', 'Evie'],
 'team2': ['Skye', 'Grohk', 'Seris', 'Inara', 'Makoa']}

In [83]:
len(matches)

37

In [84]:
matches = [match for match in matches if len(match['team1']) == 5 and len(match['team2']) == 5]
len(matches)

37

In [85]:
champions = list(set([match['Reference_Name'] for match in matchData]))
champions

['Buck',
 'Ying',
 'Cassie',
 'Imani',
 'Viktor',
 'Strix',
 'Kinessa',
 'Khan',
 'Bomb King',
 'Moji',
 'Vivian',
 'Seris',
 'Maeve',
 'Willo',
 'Tyra',
 'Fernando',
 'Barik',
 'Ash',
 'Torvald',
 'Talus',
 'Makoa',
 'Ruckus',
 'Inara',
 'Skye',
 'Pip',
 'Dredge',
 'Furia',
 'Evie',
 'Sha Lin',
 'Zhin',
 'Jenos',
 'Lian',
 "Mal'Damba",
 'Terminus',
 'Androxus',
 'Lex',
 'Grohk',
 'Drogoz',
 'Grover',
 'Koga']

In [87]:
import numpy as np

def one_hot(outcome, possibilities):
    assert outcome in possibilities
    arr = np.zeros(len(possibilities))
    arr[possibilities.index(outcome)] = 1
    return arr

one_hot('Cassie', champions)

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [160]:
def clean(match):
    winners = match['team1']
    losers = match['team2']
    x_winners = np.concatenate([one_hot(winner, champions) for winner in winners])
    x_losers = np.concatenate([one_hot(loser, champions) for loser in losers])
    x_map = one_hot(match['map'], maps)
    # shuffle team 1 and 2, mark the winner
    if r(0,1) > 0:
        return [np.concatenate((x_map, x_winners, x_losers)), np.array([1,0])]
    else:
        return [np.concatenate((x_map, x_losers, x_winners)), np.array([0,1])]

data = [clean(match) for match in matches]
X = []
Y = []
for x, y in data:
    X.append(x)
    Y.append(y)
X, Y = np.array(X), np.array(Y)
X.shape, Y.shape

((37, 412), (37, 2))

In [126]:
X = X.reshape((len(X), X[0].shape[0]))
Y = Y.reshape((len(Y), 2))

In [127]:
X.shape, Y.shape

((37, 411), (37, 2))

In [128]:
# Naive Winner Predictor

from keras.models import Sequential
from keras.layers import Dense

# Objective: predict if team 1 will win the game
model = Sequential()
model.add(Dense(1024, input_shape=(X.shape[1],), name='Input'))
model.add(Dense(64, activation='tanh', name='Hidden'))
model.add(Dense(Y.shape[1], activation='relu', name='Output'))
model.compile(optimizer='sgd', loss='mean_squared_error')

model.fit(X, Y, batch_size=1, epochs=5, validation_split=0.1, shuffle=True)

Train on 33 samples, validate on 4 samples
Epoch 1/5
33/33 [==============================] - 1s 18ms/step - loss: 0.3756 - val_loss: 0.4631
Epoch 2/5
33/33 [==============================] - 0s 6ms/step - loss: 0.1310 - val_loss: 0.4264
Epoch 3/5
33/33 [==============================] - 0s 6ms/step - loss: 0.0454 - val_loss: 0.4296
Epoch 4/5
33/33 [==============================] - 0s 5ms/step - loss: 0.0326 - val_loss: 0.4334
Epoch 5/5
33/33 [==============================] - 0s 6ms/step - loss: 0.0308 - val_loss: 0.4379


In [143]:
from tqdm import tqdm_notebook as tqdm
# Scrape data for an entire day
BATCHSIZE=10

matchData = matchData

def scrape_day(QUEUE=486, MONTH=3, DAY=14, YEAR=2019, matchData = matchData):
    matchIds = []
    print("Getting match ids...")
    for HOUR in tqdm(range(24)):
        for mm in ['00', '10', '20', '30', '40', '50']:
            _matches = api.getMatchIdsByQueue(QUEUE, datetime.date(year=YEAR, month=MONTH, day=DAY), '%02d,%s' % (HOUR,mm))
            if not _matches:
                continue
            matchIds = [match.matchId for match in _matches if not match.activeFlag]
    print('Fetching matches')
    for matchBatch in tqdm(chunks(matchIds, BATCHSIZE)):
        matchData += api.makeRequest('getmatchdetailsbatch', [','.join(matchBatch)])

In [144]:
# this can take a while be careful running it!!!!!

scrape_day()

Getting match ids...


A Jupyter Widget


Fetching matches


A Jupyter Widget

In [145]:
len(matchData)

1140

In [177]:
# MAIN TRAINING CELL
# CREATE AND TRAIN THE MODEL
# THIS IS ALMOST INSTANT ON A 1060 WITH 6GB VRAM
# AND ONLY TAKES A COUPLE SECONDS ON A i7
# THEREFORE, THIS IS NOT GOING TO BE OPTIMIZED TO ALLOW FOR RETRAINING SINCE TRAINING FROM SCRATCH IS EFFICIENT

maps = list(set([match['Map_Game'] for match in matchData]))
matches = []
for match_id in matchIds:
    mdata = [m for m in matchData if str(m['Match']) == match_id]
    match = {
        # winners
        "team1": [m['Reference_Name'] for m in mdata if m['Win_Status'] == 'Winner'],
        # losers
        "team2": [m['Reference_Name'] for m in mdata if m['Win_Status'] != 'Winner'],
        # more data
        "map": mdata[0]['Map_Game']
    }
    matches.append(match)

matches = [match for match in matches if len(match['team1']) == 5 and len(match['team2']) == 5]
champions = list(set([match['Reference_Name'] for match in matchData]))

data = [clean(match) for match in matches]
X = []
Y = []
for x, y in data:
    X.append(x)
    Y.append(y)
X, Y = np.array(X), np.array(Y)
X = X.reshape((len(X), X[0].shape[0]))
Y = Y.reshape((len(Y), 2))

# Naive Winner Predictor, but this time with 1k+ matches

# Objective: predict if team 1 will win the game
model = Sequential()
model.add(Dense(1024, input_shape=(X.shape[1],), name='Input'))
model.add(Dense(64, activation='tanh', name='Hidden'))
model.add(Dense(Y.shape[1], activation='relu', name='Output'))
model.compile(optimizer='sgd', loss='mean_squared_error')

model.fit(X, Y, batch_size=1, epochs=5, validation_split=0.1, shuffle=True)

Train on 33 samples, validate on 4 samples
Epoch 1/5
33/33 [==============================] - 0s 12ms/step - loss: 0.3840 - val_loss: 0.2937
Epoch 2/5
33/33 [==============================] - 0s 5ms/step - loss: 0.1373 - val_loss: 0.2746
Epoch 3/5
33/33 [==============================] - 0s 5ms/step - loss: 0.0463 - val_loss: 0.2516
Epoch 4/5
33/33 [==============================] - 0s 5ms/step - loss: 0.0227 - val_loss: 0.2422
Epoch 5/5
33/33 [==============================] - 0s 6ms/step - loss: 0.0175 - val_loss: 0.2395


In [178]:
# RUN THIS TOO EVERY TIME TO COMPLETE TRAINING, THE ABOVE OUTPUT IS TO VERIFY IT COMPILES
model.fit(X, Y, batch_size=1, epochs=20, validation_split=0.25, shuffle=True)

Train on 27 samples, validate on 10 samples
Epoch 1/20
27/27 [==============================] - 0s 5ms/step - loss: 6.1354e-04 - val_loss: 0.1451
Epoch 2/20
27/27 [==============================] - 0s 5ms/step - loss: 1.6009e-04 - val_loss: 0.1449
Epoch 3/20
27/27 [==============================] - 0s 5ms/step - loss: 3.9798e-05 - val_loss: 0.1444
Epoch 4/20
27/27 [==============================] - 0s 5ms/step - loss: 1.0562e-05 - val_loss: 0.1443
Epoch 5/20
27/27 [==============================] - 0s 6ms/step - loss: 3.8069e-06 - val_loss: 0.1443
Epoch 6/20
27/27 [==============================] - 0s 5ms/step - loss: 1.1582e-06 - val_loss: 0.1442
Epoch 7/20
27/27 [==============================] - 0s 6ms/step - loss: 3.0285e-07 - val_loss: 0.1442
Epoch 8/20
27/27 [==============================] - 0s 6ms/step - loss: 9.7655e-08 - val_loss: 0.1442
Epoch 9/20
27/27 [==============================] - 0s 4ms/step - loss: 3.4111e-08 - val_loss: 0.1442
Epoch 10/20
27/27 [===================

In [179]:
model.predict(np.array([X[0]]))

array([[0.9999994, 0.       ]], dtype=float32)

In [180]:
Y[0]

array([1, 0])

In [181]:
model.save('naive_winrate_predictor-6k.hd5')

In [182]:
def predict_game(mapname, team1, team2):
    # predicts probabilities of winning for both teams
    X = np.concatenate((
        one_hot(match['map'], maps),
        np.concatenate([one_hot(champion, champions) for champion in team1]),
        np.concatenate([one_hot(champion, champions) for champion in team2])
    ))
    X = np.array([X])
    result_t1, result_t2 = model.predict(X)[0]
    return {
        'Team 1': result_t1,
        'Team 2': result_t2
    }

predict_game(matches[0]['map'], matches[0]['team1'], matches[0]['team2'])

{'Team 1': 0.6888402, 'Team 2': 0.22788137}

In [183]:
Y[0]

array([1, 0])

In [184]:
maps

['Ranked Serpent Beach',
 'Ranked Frog Isle',
 'Ranked Stone Keep',
 'Ranked Jaguar Falls',
 'Ranked Fish Market',
 'Ranked Brightmarsh',
 'Ranked Frozen Guard',
 'Ranked Timber Mill',
 'Ranked Splitstone Quarry',
 'Ranked Ice Mines',
 'Ranked Ascension Peak',
 "Ranked Warder's Gate"]

In [185]:
champions

['Buck',
 'Ying',
 'Cassie',
 'Imani',
 'Viktor',
 'Strix',
 'Kinessa',
 'Khan',
 'Bomb King',
 'Moji',
 'Vivian',
 'Seris',
 'Maeve',
 'Willo',
 'Tyra',
 'Fernando',
 'Barik',
 'Ash',
 'Torvald',
 'Talus',
 'Makoa',
 'Ruckus',
 'Inara',
 'Skye',
 'Pip',
 'Dredge',
 'Furia',
 'Evie',
 'Sha Lin',
 'Zhin',
 'Jenos',
 'Lian',
 "Mal'Damba",
 'Terminus',
 'Androxus',
 'Lex',
 'Grohk',
 'Drogoz',
 'Grover',
 'Koga']

In [174]:
predict_game('Ranked Jaguar Falls', [
    'Evie',
    'Lian',
    'Ash',
    'Drogoz',
    'Jenos'
], [
    'Talus',
    'Grohk',
    'Viktor',
    'Tyra',
    'Khan'
])

{'Team 1': 0.0, 'Team 2': 0.6599741}

In [175]:
# this time im going to collect much much more data (a whole week around that day)
# btw this takes a REALLLLLLY long time so don't run this too often

scrape_day(MONTH=3, DAY=9, YEAR=2019)
scrape_day(MONTH=3, DAY=10, YEAR=2019)
scrape_day(MONTH=3, DAY=11, YEAR=2019)
scrape_day(MONTH=3, DAY=12, YEAR=2019)
scrape_day(MONTH=3, DAY=13, YEAR=2019)
scrape_day(MONTH=3, DAY=15, YEAR=2019)

Getting match ids...


A Jupyter Widget


Fetching matches


A Jupyter Widget


Getting match ids...


...

A Jupyter Widget


Getting match ids...


A Jupyter Widget


Fetching matches


A Jupyter Widget

In [176]:
len(matchData) # run this then go back to the training cell

6470